In [1]:
from anthropic import Anthropic
from anthropic.types import Message
import base64

from dotenv import load_dotenv
import os   
load_dotenv()

import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

try:
    client = Anthropic(
        api_key=os.getenv("ANTHROPIC_API_KEY"),
    )
    logger.info("Anthropic client created successfully.")
except Exception as e:
    logger.error(f"Failed to create Anthropic client: {e}")
    raise

2025-08-10 21:05:32 - __main__ - INFO - Anthropic client created successfully.


In [2]:
MODEL="claude-3-7-sonnet-20250219"
TEMPERATURE=1.0

In [19]:
def add_user_message(messages, message_content):
    if isinstance(message_content, list):
        user_message = {
            "role": "user",
            "content": message_content
        }
    else:
        user_message = {
            "role": "user",
            "content": [{"type": "text", "text": str(message_content)}]
        }

    messages.append(user_message)


def add_assistant_message(messages, message_content):
    if isinstance(message_content, Message):
        assistant_message = {
            "role": "assistant",
            "content": message_content
        }
    else:
        assistant_message = {
            "role": "assistant",
            "content": [{"type": "text", "text": message_content}]
        }

    messages.append(assistant_message)

def chat(messages, model=MODEL, temperature=TEMPERATURE, system=None, stop_sequences=None, thinking=False, thinking_budget=1024, tools=None):
    try:
        params = {
            "model": model,
            "max_tokens": 4000,
            "messages": messages,
            "temperature": temperature,
            "stop_sequences": stop_sequences,
        }
        
        if thinking:
            params["thinking"] = {
                "type": "enabled",
                "budget_tokens": thinking_budget
            }
        
        if tools:
            params["tools"] = tools

        if system:
            params["system"] = system

        return client.messages.create(**params)
    
    except Exception as e:
        logger.error(f"Chat failed: {e}")
        raise e

def extract_citation_details(results):
    citation_details = []
    for content_block in results.content:
        if hasattr(content_block, 'citations') and content_block.citations:
            for citation in content_block.citations:
                details = {
                    'cited_text': citation.cited_text,
                    'document_title': citation.document_title,
                    'start_page': citation.start_page_number,
                    'end_page': citation.end_page_number,
                    'document_index': citation.document_index
                }
                citation_details.append(details)
    return citation_details

In [4]:
def main(pdf_filepath):
    try:
        with open(pdf_filepath, "rb") as pdf_file:
            filename = os.path.basename(pdf_filepath)
            logger.info(f"filename: {filename}")

            file_bytes = base64.standard_b64encode(pdf_file.read()).decode('utf-8')
            logger.info(f"PDF file {pdf_filepath} read and encoded successfully.")

        messages = []

        add_user_message(
            messages=messages,
            message_content=[
                {
                    "type": "document",
                    "source": {
                        "type": "base64",
                        "media_type": "application/pdf",
                        "data": file_bytes,
                    },
                    "title": filename,
                    "citations": { "enabled": True }
                },
                {"type": "text", "text": "Summarize the document in one sentence"},
            ],
        )

        res = chat(messages=messages)
        logger.info("Chat response received successfully.")

        return res
    
    except Exception as e:
        logger.error(f"An error occurred in main function: {e}")
        raise e

In [5]:
results = main("assets/earth.pdf")

2025-08-10 21:05:32 - __main__ - INFO - filename: earth.pdf
2025-08-10 21:05:32 - __main__ - INFO - PDF file assets/earth.pdf read and encoded successfully.
2025-08-10 21:05:39 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-08-10 21:05:39 - __main__ - INFO - Chat response received successfully.


In [6]:
print(results)

Message(id='msg_01Uwpkz5Zdrd5HEGt2RLzEnv', content=[TextBlock(citations=[CitationPageLocation(cited_text="Earth\r\nThe Blue Marble, Apollo 17, December 1972\r\nDesignations\r\nAlternative\r\nnames\r\nThe world · The globe ·\r\nTerra · Tellus · Gaia ·\r\nMother Earth · Sol III\r\nAdjectives Earthly · Terrestrial · Terran\r\n· Tellurian\r\nSymbol and\r\nOrbital characteristics\r\nEpoch J2000\r\n[n 1]\r\nAphelion 152 097 597 km\r\nPerihelion 147 098 450 km\r\n[n 2]\r\nSemi-major axis 149 598 023 km\r\n[1]\r\nEccentricity 0.016 7086\r\n[1]\r\nOrbital period\r\n(sidereal)\r\n365.256 363 004 d\r\n[2]\r\n(1.000 017 420 96 aj)\r\nAverage orbital\r\nspeed\r\n29.7827 km/s\r\n[3]\r\nMean anomaly 358.617°\r\nInclination 7.155° – Sun's equator;\r\nEarth\r\nEarth is the third planet from the Sun and the only\r\nastronomical object known to harbor life. This is\r\nenabled by Earth being an ocean world, the only one in\r\nthe Solar System sustaining liquid surface water. Almost\r\nall of Earth's water

In [20]:
citation_details = extract_citation_details(results)
for i, details in enumerate(citation_details):
    print(f"Citation {i+1}:")
    print(f"  Text: {details['cited_text']}")
    print(f"  Document: {details['document_title']}")
    print(f"  Pages: {details['start_page']}-{details['end_page']}")
    print()

Citation 1:
  Text: Earth
The Blue Marble, Apollo 17, December 1972
Designations
Alternative
names
The world · The globe ·
Terra · Tellus · Gaia ·
Mother Earth · Sol III
Adjectives Earthly · Terrestrial · Terran
· Tellurian
Symbol and
Orbital characteristics
Epoch J2000
[n 1]
Aphelion 152 097 597 km
Perihelion 147 098 450 km
[n 2]
Semi-major axis 149 598 023 km
[1]
Eccentricity 0.016 7086
[1]
Orbital period
(sidereal)
365.256 363 004 d
[2]
(1.000 017 420 96 aj)
Average orbital
speed
29.7827 km/s
[3]
Mean anomaly 358.617°
Inclination 7.155° – Sun's equator;
Earth
Earth is the third planet from the Sun and the only
astronomical object known to harbor life. This is
enabled by Earth being an ocean world, the only one in
the Solar System sustaining liquid surface water. Almost
all of Earth's water is contained in its global ocean,
covering 70.8% of Earth's crust. The remaining 29.2% of
Earth's crust is land, most of which is located in the
form of continental landmasses within Earth's land


In [18]:
print(f"assistant response: \n{results.content[0].text}")

assistant response: 
Earth is the third planet from the Sun and the only known astronomical object to harbor life, characterized by its oceans covering 70.8% of its crust, a dynamic atmosphere primarily composed of nitrogen and oxygen, tectonic plates that create geological features, a magnetic field that protects from solar radiation, and a 4.5-billion-year history that has seen the evolution of life and human civilization, which now threatens the planet's climate and biodiversity.
